In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular, train_log, train_super_batch, train_super_batch_KL

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'1.12.1'

In [5]:
def model_saver(model, optimizer, epoch, loss, hidden_state_dict, model_name = None):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    # model_name = wandb.run.name
    if not model_name:
        model_name = "test NZ GRU saver"
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )

In [6]:
def model_saver_wandb(model, optimizer, epoch, loss, hidden_state_dict, model_name = None):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    model_name = wandb.run.name
    if not model_name:
        model_name = "test NZ GRU saver"
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )

In [7]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [8]:
def compare_model_to_bf(model:GRUNet,raceDB:Races,example_ct):
    with torch.no_grad():
        sft_max = nn.Softmax(dim=-1)
        l_sftmax = nn.LogSoftmax(dim=-1)
        nnl_loss = nn.KLDivLoss(reduction='batchmean')
        full_test_races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)))
        full_test_races_w_prices = []
        excluded, included = 0,0
        for r in full_test_races:
            if 0 in r.prices or -1 in r.prices:
                excluded+=1
            else:
                full_test_races_w_prices.append(r)
                included+=1
        print(included,excluded)

        output = l_sftmax(model(full_test_races_w_prices))
        bf_prices = torch.log(torch.tensor([x.implied_prob for x in full_test_races_w_prices ]).to('cuda:0'))
        full_classes = torch.stack([x.classes for x in full_test_races_w_prices ])

        print()

        print(f"our loss = {nnl_loss(output,full_classes)}")
        print(f"their loss = {nnl_loss(bf_prices ,full_classes)}")
        wandb.log({"our loss":nnl_loss(output,full_classes), "their loss":nnl_loss(bf_prices ,full_classes)})

In [9]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
#dog_stats_file = open( 'new gru input 2023-01.npy', 'rb')
hidden_size = 64
raceDB = build_dataset('gru_input.npy', hidden_size ,state_filter="NZ")

(1127562, 17)
(1127562, 17)
(120086, 20)
num_features_per_dog=20


  0%|          | 0/3419 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/15879 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 15879, number of unique dogs = 3419


In [10]:
date = datetime.datetime.strptime("2022-11-01", "%Y-%m-%d")
raceDB.create_test_split_date(date)

Train examples 14799, Test examples 1080


In [11]:
# dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\prediction validation 2023-01.npy', 'rb')
# hidden_size = 64
# predDB = build_dataset(dog_stats_file, hidden_size)

In [12]:
# #os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
# dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\forms\prediction_inputs_gru.npy', 'rb')
# hidden_size = 64
# predDB = build_pred_dataset(dog_stats_file, hidden_size)

In [13]:
# raceDB.create_hidden_states_dict()
# predDB.fill_hidden_states_from_dict(hidden_dict=raceDB.hidden_states_dict)
# # len(raceDB.hidden_states_dict)
# # predDB.to_cuda()

In [14]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [15]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack - NZ Testing", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = GRUNet(input_size,config['hidden_size'], output='log_softmax')
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      #criterion = nn.CrossEntropyLoss(reduction='none') #, label_smoothing=0.1 )
      criterion = nn.KLDivLoss(reduction='none')
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',threshold=0.0001, patience=50, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train_super_batch_KL(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict, model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [16]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 50, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards':5}

In [17]:
# raceDB.detach_all_hidden_states()
# raceDB.create_test_split()
# raceDB.reset_all_lstm_states()

In [18]:
# saved_dict = torch.load(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\long nsw 2000 RUN\long nsw 2000 RUN_450.pt")
# model_state_dict = saved_dict['model_state_dict']
# raceDB.fill_hidden_states_from_dict(saved_dict['db'])
# raceDB.to_cuda()
# # # predDB.fill_hidden_states_from_dict(hidden_dict=raceDB.hidden_states_dict)
# # # predDB.to_cuda()

In [19]:
raceDB.detach_all_hidden_states()

In [20]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)

(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


{'hidden_size': 64, 'batch_size': 50, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 5}
1000
{'hidden_size': 64, 'batch_size': 50, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 5}
GRUNet(
  (gru1): GRUCell(232, 64)
  (gru2): GRUCell(232, 64)
  (gru3): GRUCell(232, 64)
  (gru4): GRUCell(232, 64)
  (gru5): GRUCell(232, 64)
  (gru6): GRUCell(232, 64)
  (gru7): GRUCell(232, 64)
  (gru8): GRUCell(232, 64)
  (rl0): ReLU()
  (drop0): Dropout(p=0.3, inplace=False)
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (rl2): ReL

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpf17itu1qwandb-media\\39qbxwm9.table.json'


  0%|          | 0/107 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpf17itu1qwandb-media\\y0lnn81n.table.json'
created path



accuracy,▆▇█▇▄▁▂▃▄▅▅▃▅▆▇▆▆▇▆▇▇▆▇▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇
batch_before_backwards,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_loss,█▇▇▇▆▆▅▄▄▃▃▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
correct,▆▇█▇▄▁▂▃▄▅▅▃▅▆▇▆▆▇▆▇▇▆▇▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_loss,█▅▄▄▃▂▁▂▂▁▂▂▄▆▅▆▄▆▅▄▇▆▆▆▅▅▄▅▇▆▆▆▆▅▆▆▆▄▇▇
loss_val,█▇▆▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
multibet outlay,▁▁▂▄▅▆▇▇▇▇▇▇████████████████████████████
multibet profit,█▆▅▄▂▁▂▂▂▁▃▃▃▄▄▄▄▃▃▄▄▃▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃
profit,▄▇▇█▅▃▃▂▁▃▃▂▃▃▄▄▄▄▃▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
accuracy,0.19907


In [21]:
model_saver(model, optimizer, 450, 0.1, dataset.hidden_states_dict, model_name="long nsw 2000 RUN")

In [22]:

stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

NameError: name 'stop' is not defined

In [ ]:

dataset.create_hidden_states_dict()

created path


In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [25,50,75,100,1000]
        },
        "batch_before_backwards": {
            'values': [3,5,10,15,20,25,30,50,75,100]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_before_backwards': {'values': [3,
                                                      5,
                                                      10,
                                                      15,
                                                      20,
                                                      25,
                                                      30,
                                                      50,
                                                      75,
                                                      100]},
                'batch_size': {'values': [25, 50, 75, 100, 1000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [250]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 64},
                'l1_beta': {'value': 0.1

{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 15879},
  'hidden_size': {'value': 64},
  'epochs': {'values': [250]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [25, 50, 75, 100, 1000]},
  'batch_before_backwards': {'values': [3,
    5,
    10,
    15,
    20,
    25,
    30,
    50,
    75,
    100]}}}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="NZ_LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: n3lkddbk
Sweep URL: https://wandb.ai/nickojelly/NZ_LSTM_sweeps/sweeps/n3lkddbk


wandb: Agent Starting Run: 1ab8apie with config:
wandb: 	batch_before_backwards: 15
wandb: 	batch_size: 100
wandb: 	dropout: 0.3
wandb: 	epochs: 250
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.00033296885560502585
wandb: 	len_data: 15879
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 15, 'batch_size': 100, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.00033296885560502585, 'len_data': 15879, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
250
{'batch_before_backwards': 15, 'batch_size': 100, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.00033296885560502585, 'len_data': 15879, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNet(
  (gru1): GRUCell(232, 64)
  (gru2): GRUCell(232, 64)
  (gru3): GRUCell(232, 64)
  (gru4): GRUCell(232, 64)
  (gru5): GRUCell(232, 64)
  (gru6): GRUCell(232, 64)
  (gru7): GRUCell(232, 64)
  (gru8): GRUCell(232, 64)
  (rl0): ReLU()
  (drop0): Dropout(p=0.3, inplace=False)
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)


  0%|          | 0/250 [00:00<?, ?it/s]

...

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmppo9unnotwandb-media\\3k49g519.table.json'


  0%|          | 0/107 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmppo9unnotwandb-media\\1bnmtjek.table.json'


  0%|          | 0/107 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmppo9unnotwandb-media\\13r4h7p2.table.json'
